In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
df = pd.read_csv('Winni Reports.csv')
df.head(10)

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

In [ ]:
def rough_location(col):
    name_list = []
    directions = ['east', 'west', 'north', 'south', 'of']

    for i in col.split():
        if i not in directions:
            name_list.append(i)
    return ' '.join(name_list)

In [ ]:
df['location2'] = df['location'].map(lambda x: rough_location(x))

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df.loc[(df['wind_dir'] == 'ene')]

In [ ]:
df.at[320,'wind_dir']= 'ne'

In [ ]:
def change_fish_type(fish):
    if fish == 'small mouth bass':
        return 'smallmouth'
    elif fish == 'salmon ':
        return 'salmon'
    elif fish == 'lake trout ':
        return 'lake trout'
    return fish

df['fish_type'] = df['fish_type'].map(change_fish_type)

In [ ]:
sns.boxplot(x = df['fish_length_in']);

In [ ]:
sns.boxplot(x = df['water_depth_ft']);

In [ ]:
df.loc[df['year'] == '14:30']

In [ ]:
df.loc[df['year'] == '7:40']

In [ ]:
df = df.drop(columns = 'thermocline_depth_ft')

In [ ]:
def change_weather(condition):
    if condition == 'calm' or condition == 'partly sunny' or condition == 'slight wind':
        return 'sunny'
    elif condition == 'rainy':
        return 'raining'
    elif condition == 'cloudy':
        return 'overcast'
    return condition

df['weather'] = df['weather'].map(change_weather)

In [ ]:
df['time2'] = pd.to_datetime(df['time'])
#df['time2'] = df['time2'].dt.time

In [ ]:
#df.iloc[451, df.columns.get_loc('skunked')] = True

In [ ]:
#df['lines_in'] = pd.to_datetime(df['lines_in'])
#df['lines_in'] = df['lines_in'].dt.time

In [ ]:
#df['lines_out'] = pd.to_datetime(df['lines_out'])
#df['lines_out'] = df['lines_out'].dt.time

In [ ]:
df['duration'] =  pd.to_datetime(df['duration'])
df['minutes'] = df['duration'].dt.hour * 60 + df['duration'].dt.minute + df['duration'].dt.second/60
df.drop(columns = 'duration', inplace = True)
df.head()

## Imputing NaNs

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

In [ ]:
df['month'] = df['date'].dt.month
df.head()

In [ ]:
air_temp_dict = df.groupby('month')['air_temp_f'].mean().to_dict()
df['air_temp_f'] = df['air_temp_f'].fillna(df['month'].map(air_temp_dict))
water_temp_dict = df.groupby('month')['water_temp_f'].mean().to_dict()
df['water_temp_f'] = df['water_temp_f'].fillna(df['month'].map(water_temp_dict))
fish_length_dict = df.groupby('fish_type')['fish_length_in'].mean().to_dict()
df['fish_length_in'] = df['fish_length_in'].fillna(df['fish_type'].map(fish_length_dict))

In [ ]:
df['wind_dir'].fillna('no_wind', inplace=True)

In [ ]:
df['skunked'].fillna(False, inplace = True)

In [ ]:
df['weather'].fillna('no_weather_recorded', inplace = True)
df['time'].fillna('no_time_recorded', inplace = True)
df['fish_type'].fillna('no_fish_caught', inplace = True)
#df['fish_length_in'].fillna('no_recording', inplace = True)
#df['water_depth_ft'].fillna('no_fish_caught', inplace = True)

In [ ]:
df.rename(columns = {'minutes': 'duration_min', 'location2': 'general_loc', 'time':'time_caught'}, inplace = True)
df.drop(columns = 'time2', inplace = True)

In [ ]:
df['hour'] = df['time_caught'].map(lambda x:re.search(r'.*(?=:)',x)[0] if x != 'no_time_recorded' else 0)

In [ ]:
df['hour'] = df['hour'].astype(int)

In [ ]:
df['time_caught_bucket'] = pd.cut(df['hour'], bins=range(5, 18, 2), right=False)

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df2 = df.loc[(df['water_depth_ft'] != 'no_fish_caught')]

df2['water_depth_ft'] = df2['water_depth_ft'].astype(float)

df2['month'] = df2['month'].astype(str)

df2['fish_type_month'] = df2['fish_type']  + '_' + df2['month']

df2['month'] = df2['month'].astype(int)

df2.head()

In [ ]:
df['water_depth_ft'].value_counts()

In [ ]:
df_water_depth = df.loc[df['water_depth_ft'].isnull() == False]
df_water_depth['water_depth_ft'] = df_water_depth['water_depth_ft'].astype(float)
water_depth_dict = df_water_depth.groupby('fish_type')['water_depth_ft'].mean().to_dict()
water_depth_dict

In [ ]:
df['water_depth_ft'] = df['water_depth_ft'].fillna(df['fish_type'].map(water_depth_dict))

In [ ]:
df['water_depth_ft'].fillna(0, inplace = True)

In [ ]:
df['year'].value_counts()

In [ ]:
df.dtypes

In [ ]:
df_fish_len = df.loc[df['fish_length_in'] != 'no_recording']
df_fish_len['fish_length_in'] = df_fish_len['fish_length_in'].astype(float)
fish_len_dict = df_fish_len.groupby('fish_type')['fish_length_in'].mean().to_dict()
fish_len_dict

In [ ]:
df['fish_length_in'] = df['fish_length_in'].fillna(df['fish_type'].map(fish_len_dict))

In [ ]:
df['fish_length_in'].fillna(0, inplace = True)

In [ ]:
df['fish_length_in'] = df['fish_length_in'].astype(float)

In [ ]:
df['location'].value_counts()

In [ ]:
df.to_csv('./model_data/winni_reports.csv')

In [ ]:
df['wind_dir'].value_counts()